#### Toy example

In [ ]:
import numpy as np
from ase import Atoms
from ase.io import write, read

# create atoms object
atoms = Atoms(
    numbers = [11, 17], # NaCl
    cell = np.array([[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]),
    positions = np.array([[0, 0, 0], [1.75, 1.75, 1.75]]),
    pbc = True
)

# store target properties
formation_energy_per_atom = -1.884
forces = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
atoms.info.update({'ef_per_atom': formation_energy_per_atom})
atoms.info.update({'forces': forces})

# save 10 samples
atoms_list = [atoms] * 10
write('toy_example.xyz', atoms_list, format = 'extxyz')

In [55]:
from gnn_lib.data import atoms2data

# this is done under the hood in the build_dataloader(config, data_split) call.
graph_data = atoms2data(atoms, r_cut = 4.5, energy_key = 'ef_per_atom', forces_key='forces') 

In [56]:
graph_data.energy

tensor(-1.8840)

In [40]:
graph_data.forces

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [39]:
graph_data.numbers

tensor([11, 17])

In [31]:
graph_data.edge_index

tensor([[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
         1, 1, 1, 1],
        [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
         1, 0, 0, 0]])

In [32]:
graph_data.edge_length

tensor([3.0311, 3.0311, 3.0311, 3.5000, 3.0311, 3.5000, 3.0311, 3.5000, 3.0311,
        3.5000, 3.5000, 3.0311, 3.5000, 3.0311, 3.0311, 3.5000, 3.0311, 3.5000,
        3.5000, 3.0311, 3.5000, 3.0311, 3.5000, 3.0311, 3.5000, 3.0311, 3.0311,
        3.0311])

In [58]:
from ase.io import read
from gnn_lib.data import InMemoryDataset, atoms2data

# read saved file
atoms_list = read('toy_example.xyz', index = ':')

# create dataset from atoms_list
data_list = [atoms2data(atoms, r_cut = 4.5, energy_key = 'ef_per_atom', forces_key='forces') for atoms in atoms_list]
dataset = InMemoryDataset.from_data_list(data_list)

In [60]:
from torch.utils.data import DataLoader
from gnn_lib.data_utils import collate_fn

# create loader
loader = DataLoader(dataset,
                    collate_fn = collate_fn,
                    batch_size = 2,
                    shuffle = True,
)

for batch in loader:
    pass

In [61]:
batch.energy

tensor([-1.8840, -1.8840])